# Программирование на Python

*Алла Тамбовцева, НИУ ВШЭ*

## Управление браузером с Selenium: скачивание файлов

### Часть 1: скачиваем файлы DOC

Давайте зайдем на страницу сайта Вышки, на которой хранятся бланки заявлений для приема на работу и скачаем все бланки в формате `.doc`. В данном случае задача не очень сложная: если посмотреть на исходный код страницы, можно заметить, что для скачивания файлов необходимо просто кликнуть на ссылку, которая заканчивается расширением `.doc`. Поэтому пока не будем использовать Selenium, а вытащим из исходного кода страницы подходящие ссылки. Импортируем модуль `requests` для отправки запроса и получения исходного кода страницы и функцию `BeautifulSoup` для приведения его к более удобному виду для поиска:

In [1]:
import requests
from bs4 import BeautifulSoup

Отправляем запрос:

In [2]:
page = requests.get("https://hr.hse.ru/blanki")

В переменной `page` сейчас сохранён объект типа `Response`, скрытый объект с результатом запроса:

In [3]:
print(page, type(page))

<Response [200]> <class 'requests.models.Response'>


Из явных результатов мы видим только код 200, который означает, что запрос благополучно отправлен и ответ получен. О кодах запросов HTTP можно почитать [здесь](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status), собственно, это те коды, которые мы как пользователи можем увидеть при взаимодействии с сайтами и базами данных (известный код 404, сообщающий, что страница не найдена или код 500, всплывающий, если возникла неизвестная проблема на сервере).

Из этого объекта `page` можно извлечь различные атрибуты – составные части, которые скрыты внутри. Например, кодировку страницы:

In [4]:
print(page.encoding)

utf-8


Код запроса отдельным числом:

In [5]:
print(page.status_code)

200


Мы же извлечём исходный код страницы в виде текста:

In [ ]:
# print(page.text)

Так как работать с текстом в таком виде неудобно (только если вы не специалист по регулярным выражениям), преобразуем результат в объект `BeautifulSoup`:

In [6]:
soup = BeautifulSoup(page.text)

Теперь по этому объекту `soup` можно выполнять поиск по тэгам и атрибутам. Так как нам нужны ссылки на документы, найдём сначала все ссылки на странице. Ссылки обычно заключаются в тэг `<a>`:

In [7]:
L = soup.find_all("a")

In [8]:
L[10:12] # примеры элементов

[<a class="link link_no-underline link_dark2 fa-sidemenu__link" href="https://www.hse.ru/docs/index.html">Нормативные документы НИУ ВШЭ</a>,
 <a class="link link_no-underline link_dark2 fa-sidemenu__link" href="https://handbook.hse.ru/adm_srv">Справочник сотрудника</a>]

Теперь наша задача – отобрать элементы, которые, во-первых, являются полноценными гиперссылками (имеют атрибут `href` со ссылкой, которая куда-то перенаправляет пользователя), а, во-вторых, являются ссылками именно на документы.

Напишем цикл, который проходит по всем элементам `L`, проверяет, заканчивается ли значение в атрибуте `href` на `.doc`, и если это так, добавляет ссылку в новый список `docs`:

In [9]:
docs = []
for a in L:
    if a.get("href").endswith(".doc"):
        docs.append(a.get("href"))

Так как атрибут `href` может отсутствовать (какие-то кликабельные элементы типа кнопок имеют тэг `<a>`, а ссылки как таковой нет), напишем более общий код – добавим конструкцию `try-except`, чтобы при столкновениями с такими случаями цикл не останавливался, а просто переходил в следующему элементу:

In [10]:
# пишем try-except, чтобы код не ломался при столкновениями с такими случаями
# в итоге забираем только те ссылки, которые заканчиваются на .doc

L = soup.find_all("a")
docs = []
for a in L:
    try:
        if a.get("href").endswith(".doc"):
            docs.append(a.get("href"))
    except:
        pass

Пример нескольких ссылок:

In [11]:
print(*docs[0:10])

https://www.hse.ru/data/2021/07/13/1313008643/5%20%D0%9F%D0%9F%D0%A1_%D0%97%D0%B0%D1%8F%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BF%D1%80%D0%B8%D0%B5%D0%BC%20%D0%BF%D0%BE%20%D0%B2%D0%BD%D0%B5%D1%88%D0%BD%D0%B5%D0%BC%D1%83%20%D1%81%D0%BE%D0%B2%D0%BC.%20%D0%B0%D1%84%D1%84,%20%D0%B3%D1%80%D0%B0%D0%B6%D0%B4,%20%D0%BD%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B0_080721.doc https://www.hse.ru/data/2021/07/13/1143788942/6%20%D0%9F%D0%9F%D0%A1_%D0%97%D0%B0%D1%8F%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BF%D1%80%D0%B8%D0%B5%D0%BC%20%D0%BF%D0%BE%20%D0%B2%D0%BD%D1%83%D1%82%D1%80%20%D1%81%D0%BE%D0%B2%D0%BC,%20%D0%B3%D1%80-%D0%B2%D0%BE,%20%D0%BD%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B0_080721.doc /data/2021/07/29/1083147102/17 Заявление о приеме на работу (основное место работы, НРД) 080721.doc /data/2021/07/13/1305904881/Пенсионное-НИУ-ВШЭ.doc /data/2021/07/13/1312363400/19 Перевод 080721.doc /data/2021/07/13/1105489121/20 ППС_Заявление на перевод _080721.doc /data/2023/04/04/2025441262/20 ППС_Зая

Проблема: не все ссылки полные, то есть начинающиеся с `http` или `https`. Исправим это:

In [12]:
# если не начинается с http (https тоже под это подходит)
# приклеивай https в начало
# в любом случае добавляй ссылку doc к списку,
# либо исходную верную, либо обновленную

docs_full = []
for doc in docs:
    if not doc.startswith("http"):
        doc = "https://hse.ru" + doc
    docs_full.append(doc)

In [13]:
print(*docs_full[0:10])

https://www.hse.ru/data/2021/07/13/1313008643/5%20%D0%9F%D0%9F%D0%A1_%D0%97%D0%B0%D1%8F%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BF%D1%80%D0%B8%D0%B5%D0%BC%20%D0%BF%D0%BE%20%D0%B2%D0%BD%D0%B5%D1%88%D0%BD%D0%B5%D0%BC%D1%83%20%D1%81%D0%BE%D0%B2%D0%BC.%20%D0%B0%D1%84%D1%84,%20%D0%B3%D1%80%D0%B0%D0%B6%D0%B4,%20%D0%BD%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B0_080721.doc https://www.hse.ru/data/2021/07/13/1143788942/6%20%D0%9F%D0%9F%D0%A1_%D0%97%D0%B0%D1%8F%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BF%D1%80%D0%B8%D0%B5%D0%BC%20%D0%BF%D0%BE%20%D0%B2%D0%BD%D1%83%D1%82%D1%80%20%D1%81%D0%BE%D0%B2%D0%BC,%20%D0%B3%D1%80-%D0%B2%D0%BE,%20%D0%BD%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B0_080721.doc https://hse.ru/data/2021/07/29/1083147102/17 Заявление о приеме на работу (основное место работы, НРД) 080721.doc https://hse.ru/data/2021/07/13/1305904881/Пенсионное-НИУ-ВШЭ.doc https://hse.ru/data/2021/07/13/1312363400/19 Перевод 080721.doc https://hse.ru/data/2021/07/13/1105489121/20 ППС_Заявление на пе

Теперь осталось только прокликать все ссылки из списка с помощью Selenium, по умолчанию документы сохранятся в папку с загрузками. Импортируем модуль `webdriver` из `selenium`, а заодно и функцию `sleep` из модуля `time` для выставления задержки по времени:


In [14]:
from selenium import webdriver as wd
from time import sleep

Открываем новое окно браузера:

In [15]:
br = wd.Chrome()

Чтобы запустить скачивание файла, нужно просто поместить ссылку на него в адресную строку браузера (можете скопировать любую ссылку из `docs_full` и проверить в новой вкладке браузера). Выполним эти операции в цикле, добавив конструкцию `try-except` для учёта ошибок в случае некорректных ссылок и задержку 5 секунд между скачиваниями:

In [ ]:
# проверяем папку с загрузками
# файлы должны появляться там!

for i in docs_full:
    try:
        br.get(i)
        sleep(5)
    except:
        print("Incorrect link:", i)
        pass

### Знакомство с XPATH

**XML** (от *eXtended MarkUp Language*) – язык разметки, только в отличие от HTML не позволяет регулировать внешний вид страницы, а просто хранит данные в виде строки с удобными тэгами.

На XML-файл можно смотреть как на хранилище, откуда по запросу динамически подгружаются данные для подстановки в HTML-файл. Смысл: когда нам нужно постоянно обновлять информацию на веб-странице (каталоги товаров в магазине, данные о погоде, курсе валют), не нужно каждый раз переписывать HTML-файл, достаточно изменить XML-файл, а из него уже информация «подтянется» на страницу с помощью запроса, написанного на JavaScript.

**XPATH** (от *XML Path Language*) – язык запросов в XML-файлу, который можно использовать и для HTML тоже.

Примеры запросов (честно взяты [отсюда](https://www.w3schools.com/xml/xpath_intro.asp), очень полезный тьюториал по XPATH, у них же есть классные материалы по XML):

* `//title[@lang]`: все элементы с тэгом `<title>`, имеющие атрибут `lang`;
* `//title[@lang='en']`: все элементы с тэгом `<title>`, имеющие атрибут `lang`, равный `'en'`;
* `//title[@*]`: все элементы с тэгом `<title>`, имеющие хоть какие-нибудь атрибуты.

Поиск элементов на странице можно осуществлять и без XPATH, мы это благополучно делали, но запрос XPATH – более универсальный способ, потому что в нём можно сразу указывать и тэги, и атрибуты, даже условия или функции. Библиотека BeautifulSoup (`bs4`) поиск с помощью XPATH не поддерживает, а вот Selenium – поддерживает и даже очень. 

### Часть 2: скачиваем PDF и ZIP

Импортируем необходимую коллекцию методов `By` в дополнение к модулю `webdriver` для поиска элементов на странице с помощью Selenium:

In [16]:
from selenium.webdriver.common.by import By

Начнём новую сессию работы с браузером – новое окно браузера:

In [17]:
br = wd.Chrome()

Переходим на сайт для поиска нот (*why not?*):

In [18]:
# в открытом окне, если появится сообщение о принятии cookies 
# вручную соглашаемся, чтобы лишнее окно исчезло

br.get("https://ponotam.ru/")

Давайте для практики будем искать все необходимые элементы (поля для поиска, ссылки, кнопки) через XPATH.
Сначала нам нужно найти поле для поиска. Код для этого поля находится во фрагменте с тэгом `<input>`, при этом у этого тэга есть атрибут `id` со значением `'edit-search-block-form--2']`. Вот как это выглядит в виде запроса:

In [19]:
search = br.find_element(By.XPATH, "//input[@id='edit-search-block-form--2']")

Если бы мы искали этот элемент как раньше, могли бы просто найти его по ID (но мы не ищем лёгких путей, разбираемся с XPATH на простых примерах):

In [20]:
search = br.find_element(By.ID, "edit-search-block-form--2")

Отправляем ключевые слова для поиска – должны сами напечататься в поле на странице в новом окне браузера:

In [21]:
# метод .clear() очищает поле для ввода
# добавляем на случай, если запустим ячейку несколько раз
# чтобы ключевые слова не повторялись и не склеивались

search.clear()
search.send_keys("Happy New Year")

Теперь попробуем найти кнопку для активации поиска. Самый надёжный способ – найти её тоже по ID:

In [22]:
button = br.find_element(By.XPATH, "//input[@id='edit-submit']")

Но вариантов поиска здесь много! Посмотрим на код для кнопки:

    <input type="submit" id="edit-submit" name="op" 
    value="Поиск" class="form-submit art-button">

Мы могли бы осуществить поиск не только по `id`, но и по значению атрибутов `name`, `class` и `value`, причём как с XPATH, так и без него. Вот альтернативы (метод `get_attribute()` возвращает значение какого-нибудь атрибута, здесь мы просим «внешний» код HTML, то есть код всего элемента с кнопкой, просто чтобы проверить, что мы находим одно и то же):

In [23]:
button = br.find_element(By.NAME, "op")
print(button.get_attribute("outerHTML"))

<input type="submit" id="edit-submit" name="op" value="Поиск" class="form-submit art-button">


In [24]:
# продолжение названия после пробела нужно убрать
# по-хорошему в названии класса пробелов быть не должно
# полностью поиск с form-submit art-button не получится

button = br.find_element(By.CLASS_NAME, "form-submit") 
print(button.get_attribute("outerHTML"))

<input type="submit" id="edit-submit" name="op" value="Поиск" class="form-submit art-button">


А вот для поиска по значению в атрибуте `value` в коллекции `By` специального метода нет! Вот для таких случаев, для случаев с менее типовыми атрибутами, пригодится XPATH:

In [25]:
button = br.find_element(By.XPATH, "//input[@value='Поиск']") 
print(button.get_attribute("outerHTML"))

<input type="submit" id="edit-submit" name="op" value="Поиск" class="form-submit art-button">


Всё, кнопку точно нашли, кликаем!

In [26]:
button.click()

Перешли на страницу с результатами, давайте заберем ссылку на единственный результат по частичному тексту ссылки и кликнем на нее.

In [27]:
# внимание: тут почему-то new с маленькой буквы

link = br.find_element(By.PARTIAL_LINK_TEXT, "Happy new Year")
link.click()

Ура! Чтобы скачать zip-файл с нотами, нужно кликнуть на соответствующую иконку. Давайте найдем её через XPATH и кликнем. Так как кликать нужно на картинку, а изображения обычно хранятся в тэгах `<img>`, найдём элемент с тэгом `<img>` и атрибутом `alt` равным `'Скачать ZIP'` (`alt` – альтернативный текст, который появляется на странице, если изображение вдруг не загрузится):

In [28]:
zip_ = br.find_element(By.XPATH, "//img[@alt='Скачать ZIP']")
zip_.click()

Так как zip-архивы обычно просто скачиваются при кликании, архив должен сохраниться в папку с загрузками. С pdf-файлами всё немного сложнее. 

Найдём аналогичным образом ссылку на PDF:

In [29]:
pdf_ = br.find_element(By.XPATH, "//img[@alt='Скачать PDF']")
pdf_.click()

В браузере открылся pdf-файл, но как его автоматически скачать, неясно. Давайте откроем браузер с дополнительными опциями, чтобы PDF скачивался сразу, а не просто открывался в браузере (в Selenium есть возможности имитировать нажатие правой клавиши мышки, но здесь это не поможет – это уже не обычная веб-страница).

Вызываем опции Chrome, с которыми браузер открывается через Selenium по умолчанию:

In [30]:
options = wd.ChromeOptions()

Обновляем их – добавляем необходимые опции в виде словаря предпочтений `prefs`:
    
* `download.prompt_for_download : False`: выключаем всплывающие окна (`prompt`) с вопросами, хотим ли мы скачать файл;

* `plugins.always_open_pdf_externally : True`: включаем открытие PDF внешне (равно скачиванию), а не в новой вкладке браузера;

* `download.directory_upgrade : True`: включаем обновление папки загрузок, чтобы файл наверняка сохранился.

In [31]:
options.add_experimental_option('prefs', {
    "download.prompt_for_download": False, 
    "plugins.always_open_pdf_externally": True,
    "download.directory_upgrade": True
})

Открываем браузер с дополнением `options`:

In [32]:
br = wd.Chrome(options = options)

Осталось повторить все проделанные операции по скачиванию!

In [33]:
br.get("https://ponotam.ru/")

In [35]:
search = br.find_element(By.XPATH, "//input[@id='edit-search-block-form--2']")
search.send_keys("Happy New Year")

In [36]:
button = br.find_element(By.ID, "edit-submit")
button.click()

In [37]:
link = br.find_element(By.PARTIAL_LINK_TEXT, "Happy new Year")
link.click()

In [38]:
pdf = br.find_element(By.XPATH, "//img[@alt='Скачать PDF']")
pdf.click()